# Análisis de sentimiento 

In [1]:
import pandas as pd

df = pd.read_csv('movie_data.csv')
df.head(5)

FileNotFoundError: [Errno 2] File movie_data.csv does not exist: 'movie_data.csv'

In [2]:
df.shape

NameError: name 'df' is not defined

In [3]:
df.sentiment.value_counts()

1    25000
0    25000
Name: sentiment, dtype: int64

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 781.4+ KB


In [5]:
df.review[0]

'In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />"Murder in Greenwich" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich famil

## Crear los vectores de características

Utilizamos el método fit_transform de CountVectorizer para crear el vocabulario de la bag-of-words y transformar los enunciados en vectores.

1. The sun is shining
2. The weather is sweet
3. The sun is shining, the weather is sweet, and one and one is two


### La bolsa de palabras (bag-of-words) nos permite representar texto como vectores de características númericas. 

**Algoritmo:**

1. Crear un vocabulario de componentes léxicos únicos a partir de un conjunto único de documentos (en este caso reviews)
2. Construir un vector de características a partir de cada documento que contiene el recuento de la frecuencia en que cada palabra aparece en un documento en concreto. 


In [4]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
frases = np.array([
        'The sun is shining',
        'The weather is sweet',
        'The sun is shining, the weather is sweet, and one and one is two'])
bag = count.fit_transform(frases)

In [7]:
frases

array(['The sun is shining', 'The weather is sweet',
       'The sun is shining, the weather is sweet, and one and one is two'],
      dtype='<U64')

In [9]:
print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


In [10]:
bag

<3x9 sparse matrix of type '<class 'numpy.int64'>'
	with 17 stored elements in Compressed Sparse Row format>

In [22]:
#La matriz generada también se conoce como modelo unigrama
bag1 = bag.toarray()
print(bag1) #Podemos observar la frecuencia de cada token de las 3 frases

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


## tf-idf: term frequency-inverse document frequency

La siguiente técnica se puede traducir como frecuencia de término-frecuencia inversa de documento, se utiliza para disminuir el peso de las palabras que aparecen muchas veces en multiples documentos, la ecuación es la siguiente: 

$$\text{tf-idf}(t,d)=\text{tf (t,d)}\times \text{idf}(t,d)$$

* El producto de la frecuencia de término $(tf(t,d))$ y la frecuencia inversa de documento $(idf(t,d))$. Donde ***t*** es el número de terminos y ***d*** el número de documento.

* ***idf(t,d)*** se calcula con la siguiente ecuación:


$$\text{idf}(t,d) = \text{log}\frac{n_d}{1+\text{df}(d, t)},$$

* Donde $n_d$ es el número total de documentos 
* ***df (d,t)*** es el número de documentos ***d*** que contienen el término ***t*** 
* El logaritmo se utiliza para evitar que las bajas frecuencias de documentos no adquieran demasiado peso

### Ejemplo:
$$\text{idf}("is", d3) = log \frac{3}{1+3} = -0.287$$


$$\text{tf-idf}("is",d3)= 3 \times (-0.287) = -0.863$$

In [12]:
import math
math.log(3/4)

-0.2876820724517809

Para transformar la matriz anterior a tf-idf utilizamos la función **TfidfTransformer** de sklearn

In [6]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer()
np.set_printoptions(precision=2)
print(tfidf.fit_transform(bag).toarray())

[[0.   0.   0.58 0.58 0.35 0.   0.   0.   0.44]
 [0.58 0.   0.   0.   0.35 0.58 0.   0.   0.44]
 [0.   0.55 0.   0.   0.32 0.   0.55 0.55 0.  ]]



Las ecuaciones para idf y tf-idf implementadas en scikit-learn son: 

$$\text{idf} (t,d) = log\frac{1 + n_d}{1 + \text{df}(d, t)}$$

$$\text{tf-idf}(t,d) = \text{tf}(t,d) \times (\text{idf}(t,d)+1)$$

Scikit-learn implementa la siguiente normalización a las frecuencias (L2), que devuelve un vector de longitud 1, diviendo un vector de característica no normalizado ***v*** por su ***normativa L2***

$$v_{\text{norm}} = \frac{v}{||v||_2} = \frac{v}{\sqrt{v_{1}^{2} + v_{2}^{2} + \dots + v_{n}^{2}}} = \frac{v}{\big (\sum_{i=1}^{n} v_{i}^{2}\big)^\frac{1}{2}}$$

### Calculamos tf-idf para el termino  *is*:

In [31]:
import math
tf_is = 3
n_docs = 3
idf_is = math.log((n_docs+1) / (1+3))
tfidf_is = tf_is * (idf_is + 1)
print('tf-idf("is") = %.2f' % tfidf_is)

tf-idf("is") = 3.00


In [42]:
bag2 = np.zeros([3,9])
for i in range(bag1.shape[0]):
    for j in range(bag1.shape[1]):
        tf = bag1[i,j]
        n_docs=bag1.shape[0]
        df= np.count_nonzero(bag1[:,j])
        idf = math.log((n_docs+1) / (1+df))
        tfidf = tf * (idf + 1)
        bag2[i,j]=tfidf

In [43]:
bag2

array([[0.  , 1.  , 0.  , 1.29, 1.29, 0.  , 1.  , 0.  , 0.  ],
       [0.  , 1.  , 0.  , 0.  , 0.  , 1.29, 1.  , 0.  , 1.29],
       [3.39, 3.  , 3.39, 1.29, 1.29, 1.29, 2.  , 1.69, 1.29]])

In [52]:
#No normalizado
tfidf = TfidfTransformer(norm=None)
tfidf = tfidf.fit_transform(bag).toarray()
tfidf

array([[0.  , 1.  , 0.  , 1.29, 1.29, 0.  , 1.  , 0.  , 0.  ],
       [0.  , 1.  , 0.  , 0.  , 0.  , 1.29, 1.  , 0.  , 1.29],
       [3.39, 3.  , 3.39, 1.29, 1.29, 1.29, 2.  , 1.69, 1.29]])

In [47]:
bag2[2]/np.sqrt((bag2[2]**2).sum())

array([0.5 , 0.45, 0.5 , 0.19, 0.19, 0.19, 0.3 , 0.25, 0.19])

$$\text{tfi-df}_{norm} = \frac{[3.39, 3.0, 3.39, 1.29, 1.29, 1.29, 2.0 , 1.69, 1.29]}{\sqrt{[3.39^2+ 3.0^2+3.39^2+ 1.29^2+ 1.29^2+ 1.29^2+ 2.0^2 +1.69^2+1.29^2]}}$$$$=[0.5, 0.45, 0.5, 0.19, 0.19, 0.19, 0.3, 0.25, 0.19]$$$$\Rightarrow \text{tfi-df}_{norm}("is", d3) = 0.45$$

In [53]:
l2_tfidf = tfidf / np.sqrt(np.sum(tfidf**2))
l2_tfidf

array([[0.  , 0.13, 0.  , 0.17, 0.17, 0.  , 0.13, 0.  , 0.  ],
       [0.  , 0.13, 0.  , 0.  , 0.  , 0.17, 0.13, 0.  , 0.17],
       [0.45, 0.4 , 0.45, 0.17, 0.17, 0.17, 0.27, 0.23, 0.17]])

### Otro ejemplo

In [15]:
count = CountVectorizer()
frases = np.array([
        'The devil is furious',
        'The moon is beautifull',
        'My cat is small'])
bag = count.fit_transform(frases)
bag1 = bag.toarray()
print(count.vocabulary_)
print(bag1)
tfidf = TfidfTransformer()
np.set_printoptions(precision=2)
print(bag1)
print(tfidf.fit_transform(bag).toarray())


{'the': 8, 'devil': 2, 'is': 4, 'furious': 3, 'moon': 5, 'beautifull': 0, 'my': 6, 'cat': 1, 'small': 7}
[[0 0 1 1 1 0 0 0 1]
 [1 0 0 0 1 1 0 0 1]
 [0 1 0 0 1 0 1 1 0]]
[[0 0 1 1 1 0 0 0 1]
 [1 0 0 0 1 1 0 0 1]
 [0 1 0 0 1 0 1 1 0]]
[[0.   0.   0.58 0.58 0.35 0.   0.   0.   0.44]
 [0.58 0.   0.   0.   0.35 0.58 0.   0.   0.44]
 [0.   0.55 0.   0.   0.32 0.   0.55 0.55 0.  ]]


In [18]:
import math
tf_devil = 2
n_docs = 1
idf_devil = math.log((n_docs+1) / (1+1))
tfidf_devil = tf_devil * (idf_devil + 1)
print('tf-idf("is") = %.2f' % tfidf_devil)

tf-idf("is") = 2.00


In [20]:
bag2 = np.zeros([3,9])
for i in range(bag1.shape[0]):
    for j in range(bag1.shape[1]):
        tf = bag1[i,j]
        n_docs=bag1.shape[0]
        df= np.count_nonzero(bag1[:,j])
        idf = math.log((n_docs+1) / (1+df))
        tfidf = tf * (idf + 1)
        bag2[i,j]=tfidf

In [21]:
#No normalizado
tfidf = TfidfTransformer(norm=None)
tfidf = tfidf.fit_transform(bag).toarray()
tfidf

array([[0.  , 0.  , 1.69, 1.69, 1.  , 0.  , 0.  , 0.  , 1.29],
       [1.69, 0.  , 0.  , 0.  , 1.  , 1.69, 0.  , 0.  , 1.29],
       [0.  , 1.69, 0.  , 0.  , 1.  , 0.  , 1.69, 1.69, 0.  ]])

In [22]:
bag2

array([[0.  , 0.  , 1.69, 1.69, 1.  , 0.  , 0.  , 0.  , 1.29],
       [1.69, 0.  , 0.  , 0.  , 1.  , 1.69, 0.  , 0.  , 1.29],
       [0.  , 1.69, 0.  , 0.  , 1.  , 0.  , 1.69, 1.69, 0.  ]])